TODO

[kaggle 22 place solution]('https://www.kaggle.com/c/commonlitreadabilityprize/discussion/257302)

[github]('https://github.com/kurupical/commonlit)


configuration for almost all models:
```
epochs = 4
optimizer: AdamW
scheduler: linear_schedule_with_warmup(warmup: 5%)
lr_bert: 3e-5
batch_size: 12
gradient clipping: 0.2~0.5
reinitialize layers: last 2~6 layers
ensemble: Nelder-Mead
custom head(finally concat all)
    averaging last 4 hidden layer
    LSTM head
    vocabulary dense
hidden_states: (batch_size, vocab_size, bert_hidden_size)
  linear_vocab = nn.Sequential(
      nn.Linear(bert_hidden_size, 128),
      nn.GELU(),
      nn.Linear(128, 64),
      nn.GELU()
  )
  linear_final = nn.Linear(vocab_size * 64, 128)
  out = linear_vocab(hidden_states).view(len(input_ids), -1)) # final shape: (batch_size, vocab_size * 64)
  out = linear_final(out) # out shape: (batch_size, 128)
17 hand-made features
    sentence count
    average character count in documents
```

In [ ]:
from IPython.display import clear_output, Image
!pip install transformers
clear_output()

In [ ]:
import os
import re
import torch
import itertools
import transformers
import pandas as pd
import numpy as np
from tqdm import tqdm
from numpy import random
from torch import nn, optim
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn import model_selection
from torch.optim.lr_scheduler import LambdaLR

"""
получения информации о запущенных процессах
и использовании системы (ЦП, память, диски, сеть, датчики) в Python.
"""
import psutil

path_tr = '/content/drive/MyDrive/CommonLit/input/train.csv'
path_test = '/content/drive/MyDrive/CommonLit/input/test.csv'
path_sub = '/content/drive/MyDrive/CommonLit/input/sample_submission.csv'

SEED =13
device = 'cuda' if torch.cuda.is_available() else 'cpu'